In [4]:
import requests
import json
import math
import re
import json
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
def getProfessorList(schoolID):
    page_id = 1
    professorList = []
    while True:
        page = requests.get("http://www.ratemyprofessors.com/filter/professor/?&page=" 
                            + str(page_id) + "&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=" 
                            + str(schoolID))
        
        jsonpage = json.loads(page.content)
        professors = jsonpage['professors']
        professorList.extend(professors)
        
        if(int(jsonpage['remaining']) == 0):
            break
        else:
            page_id += 1
             
    
    df = pd.DataFrame(professorList)
    df = df.drop(df[df['tNumRatings'] == 0].index) #drop rows without responses
    df.to_json('professorsD.json')

    '''
    #save to json file
    with open('professors.json', 'w') as outfile:
        json.dump(professorList, outfile)
    '''
    
    return df
    
def getProfessorInformation(tid):
    page_id=1
    pages = []
    while True:
        url = 'http://www.ratemyprofessors.com/paginate/professors/ratings?tid='+ str(tid)+'&filter=&courseCode=&page='+str(page_id)
        page = requests.get(url);
        r_json = json.loads(page.content)
        #page_of_comments = pd.DataFrame.from_dict(r_json['ratings'], orient='columns')
        pages.extend(r_json['ratings'])
        
        if(int(r_json['remaining']) == 0):
            break
        else:
            page_id += 1
        
    df = pd.DataFrame(pages)
    prof = pList.loc[pList['tid'] == tid]
    df.insert(0,'tDept',prof['tDept'].values[0])
    df.insert(0,'tFname',prof['tFname'].values[0])
    df.insert(0,'tLname',prof['tLname'].values[0])
    df.insert(0,'tid',tid)
    
    return df

def generateAllProfInformation():
    data = []
    tids = pList['tid'].values
    
    for i in tids:
        data.append(getProfessorInformation(i))
    
    data = pd.concat(data)
    data.to_csv("profData.csv")
    
    return data

In [9]:
pList = getProfessorList(1079)

In [ ]:
data = generateAllProfInformation()